---
title: "ANOVA de Medidas Repetidas en Python"
author: "Traducido y adaptado de R por un asistente de IA"
format: live-html
pyodide: 
  packages: 
      - numpy
      - pandas
      - matplotlib
      - seaborn
      - statsmodels
      - pingouin
resources: 
    - datos
---

## ¿Qué es el ANOVA de Medidas Repetidas?

El ANOVA de medidas repetidas se utiliza para analizar datos donde los mismos sujetos se miden más de una vez. Esta prueba también se conoce como ANOVA intra-sujetos. El término "intra-sujetos" significa que los mismos individuos son medidos en la misma variable de resultado bajo diferentes puntos de tiempo o condiciones.

Por ejemplo, podrías haber medido la puntuación de autoestima (la variable de resultado) de 10 individuos en tres momentos durante un programa de intervención para determinar si su autoestima mejoró.

### Tipos de ANOVA de Medidas Repetidas

-   **ANOVA de una vía de medidas repetidas**: Una extensión de la prueba t de muestras pareadas para comparar las medias de tres o más niveles de una variable intra-sujetos (por ejemplo, tiempo).
-   **ANOVA de dos vías de medidas repetidas**: Se utiliza para evaluar simultáneamente el efecto de dos factores intra-sujetos sobre una variable de resultado continua.

## ANOVA de una vía de Medidas Repetidas en Python

Vamos a realizar un ANOVA de una vía de medidas repetidas utilizando un conjunto de datos similar al de `selfesteem` de R.

### 1. Cargar librerías y datos

Primero, instalamos y cargamos las librerías necesarias.


```{pyodide}
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
```

Usaremos `pingouin` para obtener un conjunto de datos de medidas repetidas.

```{pyodide}
# Cargamos los datos de medidas repetidas de pingouin
data = pd.read_csv('./datos/rm_anova.csv')
data.head()
```

Este conjunto de datos ya está en formato "largo", que es el adecuado para el análisis. Tenemos:
- `Subject`: El identificador del sujeto.
- `Time`: La variable intra-sujetos (por ejemplo, "pre", "post").
- `Performance`: La variable dependiente o de resultado.

### 2. Visualización de los datos

Un gráfico de cajas o de puntos puede ayudar a visualizar los cambios a lo largo del tiempo.

```{pyodide}
plt.figure(figsize=(8, 6))
sns.boxplot(x='Time', y='Performance', hue='Time', data=data, palette='viridis')
plt.title('Rendimiento a lo largo del tiempo')
plt.xlabel('Tiempo')
plt.ylabel('Rendimiento')
plt.show()

plt.figure(figsize=(8, 6))
sns.pointplot(x='Time', y='Performance', hue='Time', data=data, palette='viridis')
plt.title('Rendimiento medio a lo largo del tiempo')
plt.xlabel('Tiempo')
plt.ylabel('Rendimiento Medio')
plt.grid(True)
plt.show()
```

Los gráficos sugieren un aumento en el rendimiento a lo largo del tiempo.

### 3. Realizar el ANOVA de Medidas Repetidas

Usaremos la función `rm_anova()` de la librería `pingouin`. Necesitamos especificar:
- `dv`: La variable dependiente (`Performance`).
- `within`: La variable intra-sujetos (`Time`).
- `subject`: El identificador del sujeto (`Subject`).

```{pyodide}
import pingouin as pg

# Realizar el ANOVA de medidas repetidas
res_aov = pg.rm_anova(data=data, dv='Performance', within='Time', subject='Subject')

print(res_aov)
```

### 4. Interpretación de los resultados

La tabla de resultados nos muestra:

-   **Source**: La fuente de variación, en este caso, `Time`.
-   **p-unc**: El valor p no corregido. En nuestro ejemplo, es muy bajo (`< 0.001`), lo que indica que hay un cambio estadísticamente significativo en el `Performance` a lo largo del `Time`.

**Conclusión**: El tiempo tiene un efecto significativo en el rendimiento.

### 5. Supuesto de Esfericidad

El ANOVA de medidas repetidas tiene un supuesto adicional llamado **esfericidad**. Este supuesto se aplica cuando tienes 3 o más niveles en tu factor intra-sujetos y se refiere a que las varianzas de las diferencias entre todas las combinaciones de niveles son iguales.

`pingouin` automáticamente realiza la **prueba de esfericidad de Mauchly**:

-   **W**: El estadístico de la prueba.
-   **p-val**: El valor p de la prueba de Mauchly.
-   **sphericity**: Un booleano que indica si se cumple el supuesto (si `p-val` > 0.05).

En nuestro resultado, `sphericity` es `True` y el `p-val` es `0.393`, lo que significa que se cumple el supuesto de esfericidad y podemos confiar en el valor p no corregido (`p-unc`).

Si no se cumpliera la esfericidad (`sphericity` fuera `False`), deberíamos usar los valores p corregidos, como el de **Greenhouse-Geisser (`gg-corr`)** o el de **Huynh-Feldt (`hf-corr`)**. `pingouin` los proporciona en la misma tabla.

### 6. Pruebas Post-Hoc

Dado que el ANOVA nos dijo que hay una diferencia, pero no dónde, podemos realizar pruebas post-hoc (como comparaciones por pares con corrección de Bonferroni) para ver qué momentos son diferentes entre sí.

```{pyodide}
# Pruebas post-hoc por pares
post_hocs = pg.pairwise_tests(data=data, dv='Performance', within='Time', subject='Subject', padjust='bonf')

print(post_hocs)
```

La tabla de post-hoc nos muestra las comparaciones entre cada par de niveles de `Time`:
- `Time A` vs `Time B`: Los niveles que se comparan.
- `p-corr`: El valor p corregido.

Todas las comparaciones (`pre` vs `post`, `pre` vs `fup`, `post` vs `fup`) tienen un `p-corr` muy bajo, lo que indica que el rendimiento fue significativamente diferente en todos los momentos medidos.